In [2]:
import pandas as pd
from disambiguation.core import Database
from disambiguation.core import utils

individual_contributions_v2
newyork_combined_v2
usa_combined_v2
identities_v5
identities_adjacency_v5


In [5]:
idm = Database.IdentityManager('USA')
idm.table_name_identities

Table 'identities_v5' exists.
Table 'identities_adjacency_v5' exists.
Table 'linked_identities_v5' exists.


'identities_v5'

In [3]:
data.columns.tolist()

['recnum',
 'clustera',
 'clusterchar',
 'cmte_id',
 'ammendment',
 'rpt_tp',
 'retiringdebt',
 'microfilmloc',
 'transtype',
 'contbr_nm',
 'contbr_city',
 'contbr_st',
 'contbr_zip',
 'contbr_occupation',
 'month',
 'day',
 'century',
 'year',
 'contb_receipt_dt',
 'otherid',
 'fecrecordnum',
 'negativeamount',
 'char_137_decoded',
 'amt',
 'extracttype',
 'state',
 'submission',
 'filingperiod',
 'cluster',
 'cmte_nm',
 'cmte_treasurer_nm',
 'cmte_st1',
 'cmte_st2',
 'cmte_city',
 'cmte_st',
 'cmte_zip',
 'cmte_designation',
 'cmte_type',
 'cmte_party',
 'cmte_filing_freq',
 'cmte_interest_grp',
 'cmte_connect_org_nm',
 'cmte_candidate_id']

In [10]:
state = 'NY'
homedir = '/nfs/home/navid/data/FEC/jaygoodliffe/'
filename = homedir + '{state}_smautoclustzip.dta'.format(state = state.lower())
data = pd.read_stata(filename)
data.to_csv(homedir + '{state}_smautoclustzip.csv'.format(state = state.lower()), sep='|')
print len(data)

print len(data['cluster'].unique())

4382
328


## Functions 

In [111]:
daterange = ' TRANSACTION_DT BETWEEN "2001-01-01" AND "2004-12-30" ;'

def get_next_cluster(data):
    '''
    This generator returns a list of records from the 
    provided DataFrame. The list will correspond to one
    of the clusters in the data and the generator will
    continues until all clusters are returned.
    '''
    for cluster in  sorted(data['cluster'].unique()):
        yield data[data['cluster'] == cluster]

        
def find_match(row, retriever, verbose = False):
    '''
    Receive a row of the Goodliffe data and return a list of
    matching records from our own database.
    @param row: a row from jg data.
    @return: list of matched FEC records.
    '''
    list_tokenized_fields = ['NAME', 'ZIP_CODE', 'CONTRIBUTOR_STREET_1', 'CITY', 'STATE', 'EMPLOYER', 'OCCUPATION', 'IMAGE_NUM']
    list_auxiliary_fields = ['TRANSACTION_DT', 'TRANSACTION_AMT', 'CMTE_ID', 'ENTITY_TP', 'id']
    all_fields = list_tokenized_fields + list_auxiliary_fields
    all_fields_query = ','.join(all_fields)
    retriever.query_fields = all_fields
    
    jg_name = row['contbr_nm'].upper()
    jg_cmte_id = row['cmte_id'].upper()
    jg_city = row['contbr_city'].upper()
    jg_state = row['contbr_st'].upper()
    jg_zipcode = str(row['contbr_zip'])[:5]
    jg_microfilm = row['microfilmloc']
    jg_amount = int(row['amt'])
    jg_year = str(row['year']).rjust(2,'0')
    jg_month = str(row['month']).rjust(2,'0')
    jg_day = str(row['day']).rjust(2,'0')
    jg_century = row['century']
    
    # This is another cluster column which I'm not sure about.
    # Assuming that this is their algorithm's clustering 
    jg_cluster = row['clustera']
    
    jg_date = '{century}{year}{month}{day}'.format(century = jg_century,
                                                    year = jg_year,
                                                    month = jg_month,
                                                    day = jg_day)
    
    
    
    query = 'SELECT {fields} from usa_combined_v2 where IMAGE_NUM="{imagenum}" AND {daterange}'.format(imagenum=jg_microfilm,
                                                                                  fields=all_fields_query, daterange = daterange)

    retriever.retrieve(query=query)
    list_records = retriever.getRecords()
    list_records_new = []
    
    for r in list_records:
        matched = True
    
        
        if int(r['TRANSACTION_AMT']) != jg_amount: matched = False
        if r['TRANSACTION_DT'] != jg_date: matched = False
        if r['CITY'] != jg_city: matched = False
        if r['STATE'] != jg_state: matched = False
        if r['CMTE_ID'] != jg_cmte_id: matched = False
        if r['NAME'] != jg_name: matched = False
        if r['ZIP_CODE'][:5] != jg_zipcode: matched = False
        
        if verbose and not matched:
            print "== " + matched + " " + "="*50
            print "AMNT:: ", int(r['TRANSACTION_AMT']) , jg_amount
            print "DATE:: ", r['TRANSACTION_DT'], jg_date
            print "CITY:: ", r['CITY'], jg_city
            print "STATE:: ", r['STATE'], jg_state
            print "="*50
        
        if matched: list_records_new.append(r)
    return list_records_new
        
    
def get_records(set_rids, retriever):
    '''
    Retrieve the records with rids from a given
    set.
    '''
    if not set_rids:
        return []
    list_tokenized_fields = ['NAME', 'ZIP_CODE', 'CONTRIBUTOR_STREET_1', 'CITY', 'STATE', 'EMPLOYER', 'OCCUPATION', 'IMAGE_NUM']
    list_auxiliary_fields = ['TRANSACTION_DT', 'TRANSACTION_AMT', 'CMTE_ID', 'ENTITY_TP', 'id']
    all_fields = list_tokenized_fields + list_auxiliary_fields
    all_fields_query = ','.join(all_fields)
    retriever.query_fields = all_fields
    
    list_rids_str = '({})'.format(','.join([str(rid) for rid in set_rids]))
    
    query = 'SELECT {fields} from usa_combined_v2 where id in {list_rids_str} AND {daterange}'.format(fields=all_fields_query,
                                                                                                      daterange = daterange,
                                                                                                     list_rids_str = list_rids_str)

    retriever.retrieve(query=query)
    list_records = retriever.getRecords()
    return list_records



def get_compound_identity(rid, idm):
    '''
    Return the compound identity of rid by concatenating
    the identities of all linked identities.
    '''
    identity = idm.get_identity(rid)
    if identity is None:
        return '', []
    linked_identities = idm.get_linked_identities(identity)
    
    all_identities = linked_identities+[identity]
    compound_identity = "|".join(sorted(all_identities)) 
    return compound_identity, all_identities
    
    
    
    
    
def  get_all_related_fec_records(set_rids, idm):
    '''Get all FEC rids in the same cluster 
    as the matched records.
    @return: dict mapping rid to compound identity.
    '''
    dict_identities = {}
    set_identities = set()
    for rid in set_rids:
        compound_identity, all_identities =  get_compound_identity(rid, idm)
        
        set_identities.update(set(all_identities))
        dict_identities[rid] = compound_identity
    for identity in set_identities:
        list_rids = idm.get_ids(identity)
        
        for rid in list_rids:
            if rid not in dict_identities:
                compound_identity, all_identities =  get_compound_identity(rid, idm)
                dict_identities[rid] = compound_identity
    return dict_identities
        

    


def process_cluster(batch_index, clustera, retriever, idm):
    '''
    perform matching for all records in the cluster and
    return the full results.
    '''
    outdir = '/nfs/home/navid/data/FEC/jaygoodliffe/results/'
    counter_nomatch = 0
    counter_duplicate = 0
    list_matches_data = []
    list_columns = ['batch_index', 'jg_recnum', 'jg_cluster', 'jg_clustera', 'id', 'identity' ]
    
    # id to [(recnum,clustera), ...]. This allows us to find all records
    # matching a row, and then find all other records we clustered
    # together but which don't match any row.
    dict_matches = {}
    print "Cluster ", batch_index

#     print len(clustera)
    for j, row in clustera.iterrows():
        list_matches = find_match(row, retriever)
        
        if not list_matches:
#             print "O: No match found"
            counter_nomatch += 1
            dict_matches[(row['recnum'], row['cluster'], row['clustera'])] = [None]
            continue
            
#         print len(list_matches)
        
        # If there's only one match, accept
        if len(list_matches) == 1:
            record = list_matches[0]
            dict_matches[(row['recnum'], row['cluster'], row['clustera'])] = [record.id]
        else:
#             print counter_duplicate , " more than one match: ",  len(list_matches)
            dict_matches[(row['recnum'], row['cluster'], row['clustera'])] = [record.id for record in list_matches]
            counter_duplicate += 1

    # At this point, we have all the results in 
    # one data structures:
    # dict_matches maps (row['recnum'], row['clustera']) to a list of r_ids
    set_identities = set([idm.get_identity(rid) \
                      for jg_pair, list_rids in dict_matches.iteritems() \
                          for rid in list_rids])
    # all the FEC records matched
    set_rids = set([rid for jg_pair, list_rids in dict_matches.iteritems() \
                          for rid in list_rids])
    
    # Get all FEC records in the same cluster as the matched records.
    dict_all_fec_records = get_all_related_fec_records(set_rids, idm)

    # Set of the FEC records that didn't exist in the row but we grouped together
    # with some that were.
    set_new_rids = set(dict_all_fec_records.keys()).difference(set_rids)
    
    list_new_records = get_records(set_new_rids, retriever)
    
    
    list_matches_data = [[batch_index, key[0], key[1], key[2], rid, dict_all_fec_records[rid]]\
                        for key, list_rids in dict_matches.iteritems()\
                            for rid in list_rids]
    list_matches_data += [[batch_index, '', '', '', record.id, dict_all_fec_records[record.id]]\
                         for record in list_new_records]
    print "new ids: ", len(set_new_rids)
#     list_matches_data.append([record.id, identity, row['recnum'], row['clustera']])
    
    
    df = pd.DataFrame(list_matches_data, columns = list_columns)
    df_full = pd.merge(df,clustera, left_on='jg_recnum', right_on = 'recnum', how='left')
    outfile = outdir + 'jg_match_num_{}.csv'.format(batch_index)
    df_full.sort(['jg_cluster','identity'],inplace=True)
#     df_full['batch_id'] = batch_id
    n = len(df_full)
    m = len(df_full.columns)
    spacer = pd.DataFrame([['    ' for i in range(m)] for j in range(5)] , columns = df_full.columns)
    df_full = pd.concat([df_full,spacer])



#  break   df_full.to_csv(outfile)
#     df_full.to_html(outfile+'.html')pd.concat([df_full,
    return df_full

            

retriever = Database.FecRetriever(table_name = 'usa_combined_v2')


In [112]:
list_dfs = []
outdir = '/nfs/home/navid/data/FEC/jaygoodliffe/results/'

for batch_index, clustera in enumerate(get_next_cluster(data)):
#     if batch_index > 20: break
    df = process_cluster(batch_index, clustera, retriever, idm)
    list_dfs.append(df)
df = pd.concat(list_dfs)
df.to_csv(outdir+'jg_{state}_all_comparisons_date_restricted.csv'.format(state = state))
print "Saved to file."

KeyError: 'cluster'

In [175]:
idm.get_identity(3110336)

u'NY-466378'

In [304]:
idm.get_linked_identities('CT-22870')

[u'CT-22864', u'CT-22867', u'NY-95941', u'NY-95974', u'NY-95978']

# Load the full comparison CSV files and generate statistics 

In [12]:
def get_sizes_of_my_clusters(df):
    '''
    Receve a dataframe of records both in jay ana navid,
    and find out how many navid clusters it's split into.
    Return a sorted (descending) list of their sizes
    '''
    a = df.groupby(['identity'])
    return sorted([len(x) for x in a.groups.values()], reverse = True)
    

def get_stats_1(df):
    '''
    Receive a dataframe for a given cluster and compute
    match statistics.
    '''
    df_only_in_navid = df[df['id'].isnull()]
    df_only_in_jay = df[df['jg_recnum'].isnull()]
    df_in_both = df[df['jg_recnum'].notnull() & df['id'].notnull()]
    
    list_sizes_navid_clusters = get_sizes_of_my_clusters(df_in_both)
    
    
    
    num_only_in_navid = len(df_only_in_navid)
    num_only_in_jay = len(df_only_in_jay)
    num_in_both = len(df_in_both)
    
    
    if len(df) != num_only_in_navid + num_only_in_jay + num_in_both: 
        raise Exception("numbers don't add up")
        
    # Among the records we have in common, how close
    # are our two clusterings?
    num_distinct_identities = len(df_in_both['identity'].unique())
    
    # ratio of records found only in jay to 
    # records found in both.
    if num_in_both == 0:
        ratio_only_in_jay =  1000
        ratio_only_in_navid =  1000
    else:
        ratio_only_in_jay =  num_only_in_jay / float(num_in_both)
        ratio_only_in_navid =  num_only_in_navid / float(num_in_both)
    
    return[num_only_in_navid, num_only_in_jay, num_in_both, ratio_only_in_jay, ratio_only_in_navid, num_distinct_identities]


def get_stats_2(df):
    '''
    Receive a dataframe for a given cluster and compute
    match statistics.
    '''
    df_only_in_navid = df[df['id'].isnull()]
    df_only_in_jay = df[df['jg_recnum'].isnull()]
    df_in_both = df[df['jg_recnum'].notnull() & df['id'].notnull()]
    
    list_sizes_navid_clusters = get_sizes_of_my_clusters(df_in_both)
    
    if not list_sizes_navid_clusters: return None
    
    # ratio of the size of the larges navid cluster
    # to the total size of the cluster.
    ratio_largest_navid = list_sizes_navid_clusters[0]/float(sum(list_sizes_navid_clusters))
    
    return ratio_largest_navid
    



In [13]:
state = 'NY'

outdir = '/nfs/home/navid/data/FEC/jaygoodliffe/results/'

filename = outdir+'jg_{state}_all_comparisons_date_restricted.csv'.format(state=state)
stats_filename = outdir+'jg_{state}_all_results_date_restricted.csv'.format(state=state)
print filename

data = pd.read_csv(filename)

/nfs/home/navid/data/FEC/jaygoodliffe/results/jg_NY_all_comparisons_date_restricted.csv


# Stats, version 1 

In [130]:
columns = ['num_only_in_navid', 'num_only_in_jay', 'num_in_both', 'ratio_only_in_jay', 'ratio_only_in_navid', 'num_distinct_identities']
list_rows = []

for i in range(500):
    a = data[data['batch_index'] == str(i)]
#     print a.ix[:,:'clustera']
    if a.empty: break
    list_rows.append(get_stats_1(a))
    break

results = pd.DataFrame(list_rows,columns= columns)
results.to_csv(stats_filename)
print results

/nfs/home/navid/data/FEC/jaygoodliffe/results/jg_NY_all_comparisons_date_restricted.csv
[7, 1, 1]
   num_only_in_navid  num_only_in_jay  num_in_both  ratio_only_in_jay  \
0                  5               64            9           7.111111   

   ratio_only_in_navid  num_distinct_identities  
0             0.555556                        3  


#Stats, version 2 

In [15]:
list_rows = []
columns = ['recall']
for i in range(500):
    a = data[data['batch_index'] == str(i)]
    if a.empty: break
    list_rows.append(get_stats_2(a))
#     break

results = pd.DataFrame(list_rows,columns= columns)
results.to_csv(stats_filename)
print results
print results.mean()

       recall
0    0.777778
1    0.571429
2    0.935484
3    0.896552
4    1.000000
5    1.000000
6    0.666667
7    0.555556
8    0.500000
9    1.000000
10   0.894737
11   0.500000
12   1.000000
13   0.666667
14   0.954545
15   1.000000
16   0.952381
17   0.800000
18   0.900000
19   0.980392
20   0.800000
21   0.714286
22   0.785714
23   1.000000
24   1.000000
25   0.857143
26   0.800000
27   0.909091
28   1.000000
29   1.000000
..        ...
298  1.000000
299  1.000000
300  1.000000
301  1.000000
302  1.000000
303  1.000000
304  1.000000
305  1.000000
306  1.000000
307  1.000000
308  1.000000
309  1.000000
310  1.000000
311  1.000000
312  1.000000
313  1.000000
314  1.000000
315  1.000000
316  1.000000
317  1.000000
318  1.000000
319  1.000000
320  1.000000
321  0.923077
322  0.500000
323  0.500000
324  1.000000
325  1.000000
326  1.000000
327  1.000000

[328 rows x 1 columns]
recall    0.91753
dtype: float64


## We measured the following quantity: for each jg handcoded cluster, we find all records in our database matching a record in that cluster. We then look at how we clustered those records. A perfect agreement would mean we grouped all those records into one cluster.  However, in some cases, we group those records into multiple clusters. We take the size of the largest of those, $m$, and divide it by the total number of records in the jg cluster we were able to match, $n$. 

## A measure of our recall given their ground truth is then $m/n$, or the percentage of mutual records in that cluster that were "correctly" grouped together by our algorithm.

## For the state of NY, the average of this value over all 328 clusters is 0.92
## For the state of UT,  the average of this value over all 368 clusters  is 0.97

In [93]:
results.columns

Index([u'num_only_in_navid', u'num_only_in_jay', u'num_in_both', u'ratio_only_in_jay', u'ratio_only_in_navid', u'num_distinct_identities'], dtype='object')

In [101]:
print results['num_distinct_identities'].mean()

tmp = results['ratio_only_in_jay']
print tmp[tmp != 1000].mean()

tmp = results['ratio_only_in_navid']
print tmp[tmp != 1000].mean()


1.44915254237
1.24254157475
0.249111345028
